In [60]:
from pynq import Overlay, DefaultHierarchy, allocate, DefaultIP, MMIO
import time
import random
import numpy as np
import pynq.lib.dma
from pynq import allocate
import asyncio
import sympy

In [61]:
overlay = Overlay('/home/xilinx/pynq/overlays/Crypto_e/Crypto_e.bit')
overlay?

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0xaaaa8688>
File:            /usr/local/share/pynq-venv/lib/python3.10/site-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay /home/xilinx/pynq/overlays/Crypto_e/Crypto_e.bit. The following
attributes are available on this overlay:

IP Blocks
----------
Crypto_0             : pynq.overlay.DefaultIP
random_generator_0   : pynq.overlay.DefaultIP
processing_system7_0 : pynq.overlay.DefaultIP

Hierarchies
-----------
None

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
axi_bram_ctrl_0      : Memory
axi_bram_ctrl_1      : Memory
axi_bram_ctrl_2      : Memory
axi_bram_ctrl_3      : Memory
axi_bram_ctrl_4      : Memory
PSDDR                : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "po

In [62]:

print("Generate TF and TF_INV")
MOD = 1073750017
MOD_ROOT = 625534531
MOD_INV = 627281114
N_INV = 1073487871
TF = [1]*2048
TF_INV = [1]*2048
for i in range(1,2048):
    TF[i] = (TF[i-1]*MOD_ROOT)%MOD
    TF_INV[i] = (TF_INV[i-1]*MOD_INV)%MOD



Generate TF and TF_INV


In [89]:
class NTT_INTT():
    def __init__(self):
        self.Crypto = overlay.Crypto_0
        self.BRAM0 = overlay.axi_bram_ctrl_0
        self.BRAM1 = overlay.axi_bram_ctrl_1
        self.BRAM2 = overlay.axi_bram_ctrl_2
        self.BRAM3 = overlay.axi_bram_ctrl_3
    
        
    def LoadTF(self,TF,TF_inv):
        # 0x2000 ~ 0x3fff : Memory 'NTTTwiddleIn' (2048 * 32b)
        # Word n : bit [31:0] - NTTTwiddleIn[n]
        # 0x8000 ~ 0x9fff : Memory 'NTTTwiddleOut' (2048 * 32b)
        # Word n : bit [31:0] - NTTTwiddleOut[n]
        for i in range(2048):
            self.Crypto.write(0x2000 + i*4,TF[i])
            self.Crypto.write(0x8000 + i*4,TF_inv[i])
        self.Crypto.write(0x0018,7)
        self.Crypto.write(0x0000,1)
        
            
    def LoadData(self,data,RAMSEL):
        # 0x4000 ~ 0x7fff : Memory 'DataIn' (4096 * 32b)
        # Word n : bit [31:0] - DataIn[n]
        # 0x0010 : Data signal of RAMSel (0 or 1)
        # bit 31~0 - RAMSel[31:0] (Read/Write)
        for i in range(4096):
            self.Crypto.write(0x4000 + i*4,data[i])
        self.Crypto.write(0x0018,5)
        self.Crypto.write(0x0010,RAMSEL)
        self.Crypto.write(0x0000,1)
    
        
    def Compute(self, OP, ModIndex = 0, RAMSEL = 0):
        # 0x0018 : Data signal of OP
        # 0x0020 : Data signal of ModIndex
        # bit 31~0 - OP[31:0] (Read/Write)
        self.Crypto.write(0x0018,OP)
        self.Crypto.write(0x0020,ModIndex)
        self.Crypto.write(0x0010,RAMSEL)
        self.Crypto.write(0x0000,1)
        
        while (self.Crypto.read(0x00) & 0x4) == 0:
            pass
        
        
        
    def ReadData(self,RAMSEL):
        # 0x4000 ~ 0x7fff : Memory 'DataIn' (4096 * 32b)
        # Word n : bit [31:0] - DataIn[n]
        # 0x0010 : Data signal of RAMSel (0 or 1)
        # bit 31~0 - RAMSel[31:0] (Read/Write)
        data = [0]*4096
        self.Crypto.write(0x0018,6)
        self.Crypto.write(0x0010,RAMSEL)
        self.Crypto.write(0x0000,1)
        while (self.Crypto.read(0x00) & 0x4) == 0:
            pass
        for i in range(4096):
            data[i] = self.Crypto.read(0x4000 + i*4)
        return data
    
    def TransferData(self,RAMSEL):
        # IF RAMSEL = 0, TRANSFER DATA FROM RAMSEL 0 TO RAMSEL 1
        # IF RAMSEL = 1, TRANSFER DATA FROM RAMSEL 1 TO RAMSEL 0
        temp = [0]*4096
        temp = self.ReadData(RAMSEL)
        self.LoadData(temp,1-RAMSEL)
        
    
    def WriteDataFromBram(self,RAMSEL):
        # IF RAMSEL = 0, WRITE DATA FROM BRAM0, BRAM1
        # IF RAMSEL = 1, WRITE DATA FROM BRAM2, BRAM3        
        if RAMSEL == 0:
            for i in range(2048):
                self.Crypto.write(0x4000 + i*4,self.BRAM0.read(4*i))
            for i in range(2048):
                self.Crypto.write(0x4000 + 2048*4 + i*4,self.BRAM1.read(4*i))
        else:
            for i in range(2048):
                self.Crypto.write(0x4000 + i*4,self.BRAM2.read(4*i))
            for i in range(2048):
                self.Crypto.write(0x4000 + 2048*4 + i*4,self.BRAM3.read(4*i))
        self.Crypto.write(0x0018,5)
        self.Crypto.write(0x0010,RAMSEL)
        self.Crypto.write(0x0000,1)
                
    def ReadDataToBram(self,RAMSEL):
        # IF RAMSEL = 0, READ DATA TO BRAM0, BRAM1
        # IF RAMSEL = 1, READ DATA TO BRAM2, BRAM3  
        self.Crypto.write(0x0018,6)
        self.Crypto.write(0x0010,RAMSEL)
        self.Crypto.write(0x0000,1)     
        if RAMSEL == 0:
            for i in range(2048):
                self.BRAM0.write(i*4,self.Crypto.read(0x4000 + i*4))
            for i in range(2048):
                self.BRAM1.write(i*4,self.Crypto.read(0x4000 + 2048*4 + i*4))
        else:
            for i in range(2048):
                self.BRAM2.write(i*4,self.Crypto.read(0x4000 + i*4))
            for i in range(2048):
                self.BRAM3.write(i*4,self.Crypto.read(0x4000 + 2048*4 + i*4))
                
                
print("LOAD TWIDDLE FACTORS")
Crypto = NTT_INTT()
Crypto.LoadTF(TF,TF_INV)

LOAD TWIDDLE FACTORS


In [ ]:

iter = 1
plaintext_modulus = 65537
print("Generate Random Data")
m = [random.randint(0,plaintext_modulus) for i in range(4096)]
delta = [17 for i in range(4096)]
e = [random.randint(0,2) for i in range(4096)]
a = [random.randint(0,MOD-1) for i in range(4096)]
s = [random.randint(0,2) for i in range(4096)]

print("BEGIN TO SIM ENCRYPTION PROCESS")
print("LOAD MESSAGE and DELTA, COMPUTE MESSAGE * DELTA")
START_TIME = time.time()
# Calculate m*delta and ntt , then store in BRAM0, BRAM1
for i in range(iter):
    Crypto.LoadData(m,0)
    Crypto.LoadData(delta,1)
    Crypto.Compute(2,0,0)
    Crypto.Compute(3,0,0)
    Crypto.ReadDataToBram(0)

    Crypto.LoadData(a,0)
    Crypto.Compute(3,0,0)
    Crypto.LoadData(s,1)
    Crypto.Compute(3,0,1)
    Crypto.Compute(2,0,0)
    Crypto.TransferData(0)

    Crypto.WriteDataFromBram(0)
    Crypto.Compute(0,0,0)

    Crypto.LoadData(e,1)
    Crypto.Compute(3,0,1)
    Crypto.Compute(0,0,0)

END_TIME = time.time()
FPGA_TIME = END_TIME - START_TIME
result = Crypto.ReadData(0)

print("RESULT : ",result)
print("SIM ENCRYPTION TIME : ",FPGA_TIME)


START_TIME = time.time()
for i in range(iter):
    golden_result = [(m[i]*delta[i])%MOD for i in range(4096)]
    golden_result = sympy.ntt(golden_result,MOD)
    ntt_a = sympy.ntt(a,MOD)
    ntt_s = sympy.ntt(s,MOD)
    ntt_a_s = [ntt_a[i]*ntt_s[i]%MOD for i in range(4096)]
    golden_result = [(golden_result[i]+ntt_a_s[i])%MOD for i in range(4096)]
    ntt_e = sympy.ntt(e,MOD)
    golden_result = [(golden_result[i]+ntt_e[i])%MOD for i in range(4096)]
END_TIME = time.time()
CPU_TIME = END_TIME - START_TIME
print("GOLDEN ENCRYPTION TIME : ",CPU_TIME)
print("GOLDEN RESULT : ",golden_result)
if golden_result == result:
    print("ENCRYPTION TEST PASS")
print("SPEEDUP : ",CPU_TIME/FPGA_TIME , "X")


# Decrypt
START_TIME = time.time()
Crypto.LoadData(result,0)



START_TIME = time.time()
temp = [ntt_s[i]*ntt_a[i]%MOD for i in range(4096)]
golden_result = [(golden_result[i]-temp[i])%MOD for i in range(4096)]
golden_result = sympy.intt(golden_result,MOD)
golden_result = [(golden_result[i]//delta[i])%plaintext_modulus for i in range(4096)]
END_TIME = time.time()
CPU_TIME = END_TIME - START_TIME
print("GOLDEN DECRYPTION TIME : ",CPU_TIME)
print("GOLDEN RESULT : ",golden_result)
print("Message : ",m)


Generate Random Data
BEGIN TO SIM ENCRYPTION PROCESS
LOAD MESSAGE and DELTA, COMPUTE MESSAGE * DELTA
RESULT :  [102150777, 994201276, 135095275, 86026988, 634350258, 132454414, 973422960, 47910998, 1012240424, 900310619, 734342118, 1064451932, 353208018, 881658003, 570316662, 538057053, 302370177, 182119491, 149181199, 790997368, 578903151, 629827675, 1064860539, 243889958, 310503539, 448657307, 547018065, 314729034, 465672597, 328732937, 56034867, 990586882, 247365357, 603009870, 894087722, 48133944, 27308586, 677201533, 505995808, 789904870, 358882901, 86469413, 457143180, 940683466, 153600580, 351270308, 884663296, 419390163, 665022468, 206043457, 870466542, 1023768350, 639608164, 806643870, 223654827, 85932396, 886399367, 965832342, 1071981844, 61320624, 128868786, 798124604, 22370171, 636123644, 468470723, 375255918, 831773714, 666593607, 1065481744, 637880918, 41146565, 20951570, 758684989, 191624014, 284246066, 868718502, 451033832, 1002553433, 771721859, 239064001, 384462065, 1